In [ ]:
# Process the psv files one by one for the following tasks:
# I) Station to County
# 1) Read population data from the csv file
# 2) Get station-county mapping from county_station_list
# 3) Identify files from database folder for all counties with one or more stations

# 4) Loop over all counties:
# 4-1) Read the station files for each county 
# 4-2) Keep only needed columns: ['Station_ID', datetime, 'Latitude', 'Longitude']+
# ['temperature', 'dew_point_temperature', 'station_level_pressure', 'wind_speed', 'precipitation', 'relative_humidity', 'wet_bulb_temperature', 'sky_cover_1']
# 4-3) Round datetime to the nearest hour. Then, average weather variables for each county by datetime
# 4-4) Give the county name from the mapping. Merge with population by year-county
# 4-5) Save county data to a new csv file (a. county)

# II.
# 1) Calculate population-weighted averages of the weather variables for the state 
# 2) Save (b. WA state)
# 3) Prepare for the next step: Estimate temp density every month

In [ ]:
import os
import pandas as pd
import numpy as np
import glob

md = 'D:/OneDrive - University of Missouri/transfer_desktop/MU/2025spring_submit1'
cols_w = ['temperature', 'dew_point_temperature', 'station_level_pressure', 'wind_speed', 'precipitation', 
          'relative_humidity', 'wet_bulb_temperature', 'sky_cover_1']
## In the current study, only temperature is used for analysis. However, other met variables might be used for their correlation
#with ENSO in future studies
cols_keep = ['Station_ID', 'Station_name', 'Year', 'Month', 'Day', 'Hour', 'Minute', 'Latitude', 'Longitude'] + cols_w
cols_w[(len(cols_w)-1)] = 'skycover' # used for averaging after numerizing sky_cover

In [ ]:
# Define function to convert sky_cover to numerical values
def skycover_numerize(x):
    if x == 'CLR:00':
        return 0
    elif x == 'FEW:01':
        return 1/8
    elif x == 'FEW:02':
        return 2/8
    elif x == 'SCT:03':
        return 3/8
    elif x == 'SCT:04':
        return 4/8
    elif x == 'BKN:05':
        return 5/8
    elif x == 'BKN:06':
        return 6/8
    elif x == 'BKN:07':
        return 7/8
    elif x == 'OVC:08':
        return 1
    elif x == 'VV:09': # sky obscured
        return 1
    elif x == 'X:10':
        return np.nan
    else:
        return np.nan

In [ ]:
# Import population data
dt_pop = pd.read_csv(os.path.join(md, 'data_clean', 'population_1960_2024.csv'), usecols=['County', 'Year', 'Population'])
dt_pop = dt_pop.drop_duplicates(subset=['County', 'Year'], keep='first')

In [ ]:
#inpath = 'D:/OneDrive - University of Missouri/transfer_desktop/MU/database/GHCNh_WA'
inpath = os.path.join(md, 'data_raw', 'GHCNh_WA')
outpath = os.path.join(md, 'data_intermediate', 'GHCNh_counties25') # store the county data here
if not os.path.exists(outpath):
    os.makedirs(outpath) # create parent directory too

In [ ]:
# Import weather data - Prepare
psv_filepaths = [os.path.join(inpath, file) for file in os.listdir(inpath) if file.endswith('.psv')]

In [ ]:
# Get station-county mapping from the list
df_id = pd.read_excel(os.path.join(md, 'data_document', 'county_station_list.xlsx'), sheet_name='GHCNh', usecols=[0, 3], header=0, nrows=27)

df_id = df_id.rename(columns={'ID': 'Station_ID'})
df_id[['County', 'Station_ID']] = df_id[['County', 'Station_ID']].apply(lambda x: x.str.strip())
## remove leading and trailing spaces
df_id = df_id.drop_duplicates(subset='Station_ID', keep='first')
## delete the latter county(s) that share the same station with another county; here: Douglas - USW00094239

# Identify files for counties with one or more stations
county_stations = df_id.copy()
county_stations['filepath'] = county_stations['Station_ID'].apply(lambda x: [path for path in psv_filepaths if x in path])
county_stations = county_stations.groupby('County').agg(lambda x: x.explode().tolist()).reset_index()
## make stations and filepaths to a list for each county

In [ ]:
# Loop: Handle all counties at once
for i in np.arange(county_stations.shape[0]):
    county = county_stations.loc[i, 'County']
    paths = county_stations.loc[i, 'filepath']
    dt_cty = pd.concat([pd.read_csv(path, sep='|', usecols=cols_keep, low_memory=False) for path in paths], axis=0, ignore_index=True)
    dt_cty = dt_cty[(dt_cty['Year'] >= 2000) & (dt_cty['Year'] <= 2024)]

    # Clean weather data
    dt_cty['skycover'] = dt_cty['sky_cover_1'].apply(skycover_numerize) # numerize sky_cover
    dt_cty['precipitation'] = dt_cty['precipitation'].apply(lambda x: 0 if np.isnan(x) else x) # replace missing precipitation with 0
    dt_cty['relative_humidity'] = dt_cty['relative_humidity'].apply(lambda x: 109.9876 if x > 110 else x) # relative humidity can't be > 110
    dt_cty['dew_point_temperature'] = dt_cty['dew_point_temperature'].apply(lambda x: 20.9876 if x > 21 else x) # dew_point_temperature can hardly be > 21
    dt_cty['wet_bulb_temperature'] = dt_cty['wet_bulb_temperature'].apply(lambda x: 35.9876 if x > 36 else x) # wet_bulb_temperature can hardly be > 36
    dt_cty['temperature'] = dt_cty['temperature'].apply(lambda x: 47.9876 if x > 48 else x) # temperature can't be higher than 48.9
    # Average weather variables by datetime
    dt_cty['datetime'] = pd.to_datetime(dt_cty[['Year', 'Month', 'Day', 'Hour', 'Minute']]).dt.round('h')
    ## round the datetime to the nearest hour # previously: dt.floor.('h')
    dt_cty = dt_cty.groupby(['datetime'])[cols_w].mean().reset_index()
    dt_cty = dt_cty.dropna(subset=['temperature']) # drop rows with missing temperature because all models need temperature
    
    # Merge with population
    dt_cty['County'] = county
    dt_cty['Year'] = dt_cty['datetime'].dt.year
    dt_cty = dt_cty.merge(dt_pop, on=['Year', 'County'], how='left')
    dt_cty.to_csv(os.path.join(outpath, f'{county}_2000-2024.csv'), index=False)
    print(county)

In [ ]:
# Define function to calculate population-weighted average
def weighted_avg(df, xcols, w):
    result = {}

    for col in xcols:
        valid_mask = df[col].notna()  # Mask for non-missing values in this column
        wt_sum = (df[col] * df[w]).where(valid_mask, 0)  # Multiply only valid values
        sumofwts = df[w].where(valid_mask, 0)  # Sum only valid weights

        group_wt_sum = wt_sum.groupby(df['datetime']).sum()
        group_sumofwts = sumofwts.groupby(df['datetime']).sum()

        result[col] = group_wt_sum / group_sumofwts  # Compute weighted avg

    return pd.DataFrame(result).reset_index()

In [ ]:
county_files = glob.glob(os.path.join(outpath, '*.csv'))
dt_counties = pd.concat([pd.read_csv(file) for file in county_files], ignore_index=True)
dt_counties['datetime'] = pd.to_datetime(dt_counties['datetime'])
wt_avgs = weighted_avg(dt_counties, cols_w, 'Population')
wt_avgs.to_csv(os.path.join(md, 'data_clean', 'WA_weather_1990-2024.csv'), index=False)

In [ ]:
# Prepare for regression - Estimate kernel density

In [ ]:
from scipy.stats import gaussian_kde

def kde_month_t(df):
    df['f_r'] = None
    for month in df['yyyymm'].unique():
        month_data = df.loc[df['yyyymm']==month, 'temperature'].values
        
        if len(month_data) < 200:
            density_est = np.nan
        else:
            kde = gaussian_kde(month_data)
            density_est = kde(month_data)
        
        df.loc[df['yyyymm']==month, 'f_r'] = density_est
    return df

def get_density(df):
    # Add year-month variable and compute density
    df['datetime00'] = pd.to_datetime(df['datetime00'])
    df['Year'] = pd.to_datetime(df['datetime00']).dt.year
    df['Month'] = pd.to_datetime(df['datetime00']).dt.month
    df['yyyymm'] = df['Year'].astype(str) + '-' + df['Month'].astype(str).str.zfill(2)
    return kde_month_t(df)    

In [ ]:
dt_wa = get_density(wt_avgs)
dt_wa.to_csv(os.path.join(md, 'data_clean', 'WA_wt_density.csv'), index=False)